<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 7: Correlations</h1>


<a name='section_7_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_1">L7.1 Understanding Best Fit (Revisited)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_1">L7.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_2">L7.2 Minimizing on a Surface (1D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_2">L7.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_3">L7.3 Minimizing on a Surface (2D Scan)</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_3">L7.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_4">L7.4 Correlations Between Fit Parameters: Part 1</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_4">L7.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_7_5">L7.5 Correlations Between Fit Parameters: Part 2</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_7_5">L7.5 Exercises</a></td>
    </tr>
</table>



<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

See the source and attribution information below:

>data: data/L05/events_a4_1space.dat, data/L05/events_a8_1space.dat <br>
>source: https://www.auger.org/index.php/science/data <br>
>source : https://arxiv.org/abs/1709.07321 <br>
>attribution: Pierre Auger Collaboration, arXiv:1709.07321v1 <br>
>license type: https://creativecommons.org/licenses/by-sa/4.0/ 

In [ ]:
#>>>RUN: L7.0-runcell00

#importing data from git repository

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L05' >> .git/info/sparse-checkout
!git pull origin main

In [ ]:
#>>>RUN: L7.0-runcell01

# for Colab users
!pip install lmfit

In [ ]:
#>>>RUN: L7.0-runcell02

import numpy as np                 #https://numpy.org/doc/stable/
from scipy import optimize as opt  #https://docs.scipy.org/doc/scipy/reference/optimize.html
import matplotlib.pyplot as plt    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import math                        #https://docs.python.org/3/library/math.html
import csv                         #https://docs.python.org/3/library/csv.html
import lmfit                       #https://lmfit.github.io/lmfit-py/ 
import scipy.stats as stats        #https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy.optimize.zeros import RootResults        #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.RootResults.html
from scipy.optimize.optimize import OptimizeResult  # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.OptimizeResult.html
from typing import Callable, Tuple #https://docs.python.org/3/library/typing.html#typing.Callable
                                   #https://docs.python.org/3/library/typing.html
import numpy.linalg as la          #https://docs.scipy.org/doc/scipy/reference/linalg.html
from matplotlib.patches import Ellipse  # https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.patches.Ellipse.html


In [ ]:
#>>>RUN: L7.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_7_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.1 Understanding Best Fit (Revisited)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_0) | [Exercises](#exercises_7_1) | [Next Section](#section_7_2) |


In [ ]:
#>>>RUN: L7.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_01.html', width=970, height=550)

In [ ]:
#>>>RUN: L7.1-runcell01

import scipy.stats as stats

sigma = 1
print(stats.norm.cdf(sigma)-stats.norm.cdf(-sigma))

def pval(iVal):
    return stats.norm.cdf(iVal)-stats.norm.cdf(-iVal)

def chi2Val(iGausSigma,iNDOF):
    val=stats.chi2.ppf(pval(iGausSigma),iNDOF)
    return val

print(chi2Val(1,2))
#print(chi2Val(1,1))
#print(chi2Val(2,1))
#print(chi2Val(3,1))
#print(chi2Val(4,1))

<a name='exercises_7_1'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_1) | [Next Section](#section_7_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.1.1</span>

Compute the Chi-square value for 2$\sigma$ and 2 degrees of freedom. Enter your answer as a number with precision 1e-2. 


In [ ]:
#>>>EXERCISE: L7.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.


<a name='section_7_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.2 Minimizing on a Surface (1D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_1) | [Exercises](#exercises_7_2) | [Next Section](#section_7_3) |


In [ ]:
#>>>RUN: L7.2-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_02.html', width=970, height=550)

In [ ]:
#>>>RUN: L7.2-runcell01

############ This is all code from a previous lesson
import numpy as np
import csv
import math
import lmfit
import matplotlib.pyplot as plt
from scipy import optimize as opt

def rad(iTheta):
    return iTheta/180. * math.pi

def rad1(iTheta):
    return iTheta/180. * math.pi-math.pi

def load(label):
    dec=np.array([])
    ra=np.array([])
    az=np.array([])
    with open(label,'r') as csvfile:
        plots = csv.reader(csvfile,delimiter=' ')
        for pRow in plots:
            if '#' in pRow[0] or pRow[0]=='':
                continue
            dec = np.append(dec,rad(float(pRow[2])))
            ra  = np.append(ra,rad1(float(pRow[3])))
            az  = np.append(az,rad(float(pRow[4])))
    return dec,ra,az

def prephist(iRA):
    y0, bin_edges = np.histogram(iRA, bins=30)
    x0 = 0.5*(bin_edges[1:] + bin_edges[:-1])
    y0 = y0.astype('float')
    return x0,y0,1./(y0**0.5)

label8='data/L05/events_a8_1space.dat'

dec,ra8,az=load(label8)
xhist,yhist,xweights=prephist(ra8)


########## Tlast fit code

def fnew(x,a,b):
    pVal=b*np.sin(x)
    return a+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=10)
result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()


In [ ]:
#>>>RUN: L7.2-runcell02

#This is the new code
def chi2(iX):
    #Note that this function depends on xhist and yhist being defined already.
    #This is bad practice in general, but we do it here for convenience.
    #After all, this code isn't reused elsewhere.
    #MAKE SURE YOU RUN prephist(ra8) to define yhist
    assert len(iX) == 2
    lTot=0
    for val in range(len(yhist)):
        xtest=fnew(xhist[val],iX[0],iX[1])
        lTot += (1./(xtest+1e-5))*(yhist[val]-xtest)**2
    return lTot

#First we minimize
x0 = np.array([1000,10]) # initial conditions
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

In [ ]:
#>>>RUN: L7.2-runcell03

#Scan near the minimum of each value
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)

#Now let's fix one parameter at the minimum, and profile the other
plt.plot(x, chi2([x,sol.x[1]]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("a-value")
plt.ylabel("$\chi^{2}$")
plt.show()

#Now for the other parameter
plt.plot(y, chi2([sol.x[0],y]),label='chi2');
plt.axhline(sol.fun+1, c='red')
plt.xlabel("b-value")
plt.ylabel("$\chi^{2}$")
plt.show()

In [ ]:
#>>>RUN: L7.2-runcell04

######## Now let's use a numerical solver to find the points at which a function crosses zero (root solver)
def chi2minX(xval, delta_chi2=1):
    val=chi2([xval,sol.x[1]])
    minval=chi2(sol.x) + delta_chi2
    return val-minval

def chi2minY(yval, delta_chi2=1):
    val=chi2([sol.x[0],yval])
    minval=chi2(sol.x) + delta_chi2
    return val-minval

def chi2uncX(sol):
    solX1=opt.root_scalar(chi2minX,bracket=[sol.x[0], sol.x[0]*1.02],method='brentq')
    solX2=opt.root_scalar(chi2minX,bracket=[sol.x[0]*0.98, sol.x[0]],method='brentq')
    print("a:",sol.x[0],"+/-",abs(solX2.root-solX1.root)/2.)
    print("Reminder the Poisson uncertainty would be:",math.sqrt(sol.x[0]/40))
    return solX1, solX2

def chi2uncY(sol):
    solY1=opt.root_scalar(chi2minY,bracket=[sol.x[1],    sol.x[1]*1.2],method='brentq')
    solY2=opt.root_scalar(chi2minY,bracket=[sol.x[1]*0.8, sol.x[1]],method='brentq')
    print("b:",sol.x[1],"+/-",abs(solY2.root-solY1.root)/2.)
    return solY1, solY2

solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)

<a name='exercises_7_2'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_2) | [Next Section](#section_7_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.2.1</span>

What are the 2$\sigma$ bounds (that is, 95.45% confidence interval values) of $a$ and $b$ in the fit above? Enter your answer as a list of numbers with precision 1 (the nearest whole number): `[a_lower, a_upper, b_lower, b_upper]`

In [ ]:
#>>>EXERCISE: L7.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#For a given parameter, assume everything else is determined, so we're
# working with 1 degree of freedom.
# First, we want to know the chi-square value at which 2 standard deivations (95.45% of the probability)
# in the chi-square distribution is for values less extreme than that value.
ndof=1
pval=0.9545
val = stats.chi2.ppf(pval,ndof)
print("Delta Chi-square:", val)

def minXfunc(x):
    return chi2minX(x, delta_chi2=val)

def minYfunc(x):
    return chi2minY(x, delta_chi2=val)

def chi2unc(sol, sol_index, min_func, unc_prop_guess):
     #insert code here
    return valmin,valmax

a1, a2 = chi2unc(sol, 0, minXfunc, 0.08)
b1, b2 = chi2unc(sol, 1, minYfunc, 0.4)

print(f"a bounds: [{a1}, {a2}]")
print(f"b bounds: [{b1}, {b2}]")

<a name='section_7_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.3 Minimizing on a Surface (2D Scan)</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_2) | [Exercises](#exercises_7_3) | [Next Section](#section_7_4) |


In [ ]:
#>>>RUN: L7.3-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_03.html', width=970, height=550)

In [ ]:
#>>>RUN: L7.3-runcell01

#define the 2D X and Y grid
x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100) #grid in x
y = np.linspace(sol.x[1]*0.5,sol.x[1]*1.5, 100)#grid in y
X, Y = np.meshgrid(x, y) #2d grid

# For z coordinate, evaluate chi2 at each x,y point in the grid.
# note understanding this one-liner itself isn't too important
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)

#and plot
def plotColorsAndContours(X,Y,Z):
    fig, ax = plt.subplots(1, 1)
    c = ax.pcolor(X,Y,Z,cmap='RdBu')
    cb=fig.colorbar(c, ax=ax)
    plt.xlabel("a")
    plt.ylabel("b")
    cb.set_label("$\chi^{2}$")
    #Now let's plot the contours of Delta chi^2
    levels = [0.1,1,2.3,4,6.18,9, 16, 25, 36, 49, 64, 81, 100]
    for i0 in range(len(levels)):
        levels[i0] = levels[i0]+sol.fun
    c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
    #plt.show()
    
plotColorsAndContours(X,Y,Z)

In [ ]:
#>>>RUN: L7.3-runcell02

#Let's plot the uncertainties  from hess_inv
print(np.sqrt(2*sol.hess_inv))
#the diagonals are approximately the errors

#Make a the expression in the above equation x and x0 are 2 vectors
def quadratic2D(x,x0,sigma0):
    lVals=x-x0
    lVals=(lVals**2)/(sigma0)/sigma0
    return np.sum(lVals)

plotColorsAndContours(X,Y,Z)

#Now plot the ellipse in 3D
def plotEllipse(sigx,sigy):
    levels = [0.1,1,2.3,4,6.18,9, 16, 25, 36, 49, 64, 81, 100]
    ZQ = np.array([quadratic2D([x,y],sol.x,[sigx,sigy]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
    c = plt.contour(X, Y, ZQ, levels,colors=['red', 'blue', 'yellow','green'],linestyles='dashed')

sigx=(solX2.root-solX1.root)/2.
sigy=(solY2.root-solY1.root)/2.
plotEllipse(sigx,sigy)
plt.show()

<a name='exercises_7_3'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_3) | [Next Section](#section_7_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.3.1</span>

When we allow 2 parameters to vary, for a given confidence level (e.g. 1$\sigma$) we end up with a confidence ellipse containing parameter values outside the confidence intervals on the individual parameters alone. Compute the 1$\sigma$ (68.27%) confidence interval bounds for the parameter $a$, based on this ellipse from floating both $a$ and $b$.

Enter your answer as a list of number with precision 1 (nearest whole number): `[a_lower, a_upper]`

Why do we typically quote the uncertainty from the 1D variation?

In [ ]:
#>>>EXERCISE: L7.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

# We can follow the same procedure for finding the confidence interval based on 1D
# but for 1-sigma we now use 2.3 
# Use the code from the solution to Ex. 7.2.1,
ndof=#insert value here
pval=0.6827#1 sigma p-value
val = stats.chi2.ppf(pval,ndof) # 2 DoF this time!
print("Delta Chi-square:", val)

def minfunc(x):
    return chi2minX(x, delta_chi2=val)

a1, a2 = #YOUR CODE HERE

print(f"a bounds: [{a1}, {a2}]")

<a name='section_7_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.4 Correlations Between Fit Parameters: Part 1</h2>  

| [Top](#section_7_0) | [Previous Section](#section_7_3) | [Exercises](#exercises_7_4) | [Next Section](#section_7_5) |


In [ ]:
#>>>RUN: L7.4-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L07/slides_L07_04.html', width=970, height=550)

In [ ]:
#>>>RUN: L7.4-runcell01

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=1000)

result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

x0 = np.array([1000,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
plotColorsAndContours(X,Y,Z)
plt.show()

In [ ]:
#>>>RUN: L7.4-runcell02

plotColorsAndContours(X,Y,Z)
solX1, solX2 = chi2uncX(sol)
solY1, solY2 = chi2uncY(sol)
sigx=(solX2.root-solX1.root)/2.
sigy=(solY2.root-solY1.root)/2.
plotEllipse(sigx,sigy)
plt.show()


<a name='exercises_7_4'></a>     

| [Top](#section_7_0) | [Restart Section](#section_7_4) | [Next Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.4.1</span>

Run the previous fit with just a regular linear fit given by $f(x) = ax + b$. How does the $\chi^{2}$ change? How do the correlations between the variables change? Choose the best option from the following:

- $\chi^{2}$ increases and the correlations increase
- $\chi^{2}$ decreases and the correlations increase
- $\chi^{2}$ stays approximately the same and the correlations increase
- $\chi^{2}$ increases and the correlations decrease
- $\chi^{2}$ decreases and the correlations decrease
- $\chi^{2}$ stays approximately the same and the correlations decrease
- $\chi^{2}$ increases and the correlations stay approximately the same
- $\chi^{2}$ decreases and the correlations stay approximately the same
- $\chi^{2}$ stays approximately the same and the correlations stay approximately the same

In [ ]:
#>>>EXERCISE: L7.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.



<a name='section_7_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L7.5 Correlations Between Fit Parameters: Part 2</h2>     

| [Top](#section_7_0) | [Previous Section](#section_7_4) | [Exercises](#exercises_7_5) |


In [ ]:
#>>>RUN: L7.5-runcell01

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

model  = lmfit.Model(fnew)
p = model.make_params(a=1000,b=1000)

result = model.fit(data=yhist,x=xhist, params=p, weights=xweights)
lmfit.report_fit(result)
plt.figure()
result.plot()
plt.show()

x0 = np.array([1000,1000])
ps = [x0]
sol=opt.minimize(chi2, x0)
print(sol)

x = np.linspace(sol.x[0]*0.99,sol.x[0]*1.01, 100)
y = np.linspace(sol.x[1]*0.99,sol.x[1]*1.01, 100)
X, Y = np.meshgrid(x, y)
Z = np.array([chi2([x,y]) for (x,y) in zip(X.ravel(), Y.ravel())]).reshape(X.shape)
plotColorsAndContours(X,Y,Z)
plt.show()


In [ ]:
#>>>RUN: L7.5-runcell02

print(np.sqrt(2*sol.hess_inv))
#The diagonals are the uncertainty lmfit quotes

#Really the best way to do this is to get the eigen values using an linear algebra problem
import numpy.linalg as la
w, v=la.eig(2*sol.hess_inv)
print("values",w,"vectors",v)

#Now let's plot the eigenvectors
from matplotlib.patches import Ellipse
def get_cov_ellipse(cov, centre, nstd, **kwargs):
    """
    Return a matplotlib Ellipse patch representing the covariance matrix
    cov centred at centre and scaled by the factor nstd.

    """
    # Find and sort eigenvalues and eigenvectors into descending order
    eigvals, eigvecs = np.linalg.eigh(cov)
    order = eigvals.argsort()[::-1]
    eigvals, eigvecs = eigvals[order], eigvecs[:, order]

    # The anti-clockwise angle to rotate our ellipse by 
    vx, vy = eigvecs[:,0][0], eigvecs[:,0][1]
    theta = np.arctan2(vy, vx)

    # Width and height of ellipse to draw
    width, height = 2 * nstd * np.sqrt(eigvals)
    return Ellipse(xy=centre, width=width, height=height,angle=np.degrees(theta), **kwargs)

err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x,1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
levels = [0.1,1,2.3,4,9, 16, 25, 36, 49, 64, 81, 100]
for i0 in range(len(levels)):
    levels[i0] = levels[i0]+sol.fun
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.show()

In [ ]:
#>>>RUN: L7.5-runcell03

#Now let's get the correlation C(a,b) (see below)
w, v=np.linalg.eig(2*sol.hess_inv)
print("c(a,b)",v[0,1]/v[0,0])
print("A deceptively wrong way to get correlation: since its not normalized",sol.hess_inv[0,1]/sol.hess_inv[0,0])

In [ ]:
#>>>RUN: L7.5-runcell04

import lmfit

def fnew(x,a,b):
    pVal=b*(1-x)
    return a*x+pVal

#Randomly sample points in the above range
def maketoy(iy):
    toy=np.array([])
    #go through the y-values and Poisson fluctuate
    for i0 in range(len(iy)):
        pVal = np.random.normal (iy[i0],np.sqrt([iy[i0]]))
        toy = np.append(toy,float(pVal))
    return toy

def fittoy(ibin,iy):
    #generate toy
    toy=maketoy(iy)
    #now fit
    model  = lmfit.Model(fnew)
    p = model.make_params(a=1000,b=10)
    xweights=np.array([])
    #setup poison weight
    for i0 in range(len(toy)):
        xweights = np.append(xweights,1./math.sqrt(toy[i0]))
    result = model.fit(data=toy,x=ibin, params=p, weights=xweights)
    return result.params["a"].value,result.params["b"].value

ntoys=1000
lAs=np.array([])
lBs=np.array([])
for i0 in range(ntoys):
    pA,pB=fittoy(xhist,yhist)
    lAs = np.append(lAs,pA)
    lBs = np.append(lBs,pB)

err_ellipse=get_cov_ellipse(2*sol.hess_inv,sol.x, 1)
fig, ax = plt.subplots(1, 1)
c = ax.pcolor(X,Y,Z,cmap='RdBu')
fig.colorbar(c, ax=ax)
c = plt.contour(X, Y, Z, levels,colors=['red', 'blue', 'yellow','green'])
ax.add_artist(err_ellipse)
plt.plot(lAs,lBs,c='black',marker='.',linestyle = 'None')
plt.show()

In [ ]:
#>>>RUN: L7.5-runcell05

#Now let's run a. linear regression
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

print("A:",lAs.mean(),"+/-",lAs.std())
print("B:",lBs.mean(),"+/-",lBs.std())
print("Cov:",covariance(lAs,lBs),"A Variance:",variance(lAs),"B Variance:",variance(lBs))
print("Check with Hessian:",2*sol.hess_inv)
print("Cor:",covariance(lAs,lBs)/math.sqrt(variance(lAs)*variance(lBs)),"A Variance:",1.,"B Variance:",1.)

<a name='exercises_7_5'></a>   

| [Top](#section_7_0) | [Restart Section](#section_7_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-7.5.1</span>

Repeat the previous analysis using the uncorrelated fit, $f(x) = a x + b$. Specifically, plot the ellipse and compute the uncertainties in $a$ and $b$? Do they correspond with lmfit? 


In [ ]:
#>>>EXERCISE: L7.5.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

